In [1]:
import sys

from dspy.retrieve.opensearch import OpenSearchRM

sys.path.append('.')

import dspy

model = dspy.OpenAI(model='TheBloke/leo-hessianai-13B-chat-AWQ', api_base="http://hal9000:1897/v1", api_key="fu", stop="\n\n")

retriever = OpenSearchRM()

In [2]:
dspy.settings.configure(lm=model,rm=retriever)

In [3]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


In [4]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question="Welche ist die Hauptstadt von Frankreich?")
print(f"Predicted Answer: {pred.answer}")

Predicted Answer: Paris


In [5]:
model.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: Welche ist die Hauptstadt von Frankreich?
Answer: Paris





In [14]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question="Welche ist die Hauptstadt von Frankreich?")
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

Thought: We know that Paris is the capital city of France, so if someone asks us which is the capital city of France, we should answer "Paris".
Predicted Answer: Paris


In [15]:
model.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: often between 1 and 5 words

---

Question: Welche ist die Hauptstadt von Frankreich?
Reasoning: Let's think step by step in order to find out the capital city of France. We know that Paris is the capital city of France, so if someone asks us which is the capital city of France, we should answer "Paris".
Answer: Paris





In [9]:
question = "Wie hoch sind die Zinsen für ein KfW Studiumskredit?"

retrieve = dspy.Retrieve()
topK_passages = retrieve(question).passages

print(f"Top {retrieve.k} passages for question: {question} \n", '-' * 30, '\n')

for idx, passage in enumerate(topK_passages):
    print(f'{idx+1}]', passage, '\n')

Top 3 passages for question: Wie hoch sind die Zinsen für ein KfW Studiumskredit? 
 ------------------------------ 

1] Wie viele KfW-Studienkreditnehmenden zahlen derzeit einen Zinssatz von über 7 % (bitte auch prozentualen Anteil an Gesamtzahl der Studienkreditnehmenden nennen)?  9. Auf welche Gesamtsumme belaufen sich die ausstehenden KfW- Studienkredite (bitte aufschlüsseln nach Rückzahlungs-, Karenz- und Auszahlungsphase sowie gruppieren in „unterliegen Zinssatz von unter 5 %”, „unterliegen Zinssatz zwischen 5 % und 7 %” und „unterliegen Zinssatz von über 7 %”)? 10. Gibt es Prognosen von Seiten der Bundesregierung oder Prognosen, auf die sich die Bundesregierung stützt hinsichtlich der Zahl der KfW- Studienkreditnehmenden in den nächsten fünf Jahren, und wenn ja, wie sehen diese aus? 11. Wie hoch war der Gewinn bzw. Verlust, den die KfW in den vergangenen zehn Jahren lediglich mit dem KfW-Studienkredit gemacht hat (bitte nach Jahren auflisten)? 12.  

2] Welchen Zinssatz haben Per

In [10]:
class GenerateAnswer(dspy.Signature):
    """Answer the question with one or more whole sentences. Base your answer solely on the context. Use the same language used in the question for your answer."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="one or more whole sentences")

In [11]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [32]:
from dspy.datasets.dataset import Dataset


class TrainingSet(Dataset):

    def __init__(self, data: dict[str,str], **kwargs):
        super().__init__(**kwargs)
        

        self._train = [dspy.Example(**x).with_inputs('question') for x in data]


trainset = TrainingSet(
    [
        {"question":"Wie werden dänische Mittelnamen im deutschen Namensrecht gehandhabt?"},#, "answer":"Dänische Mittelnamen werden im deutschen Namensrecht als Teil des Familiennamens behandelt. Sie können nur gebildet werden, indem ein Vorname als Teil des Doppelnamens verwendet wird, den das Kind von einem Elternteil erbt. Ein dänischer Mittelnamen kann nicht als separater Name neben dem Hauptnamen existieren. Wenn ein Elternteil einen Doppelnamen hat, kann der Mittelnamen jedoch als erster Teil des Doppelnamens des Kindes verwendet werden."},
        {"question":"Welchen Zusammenhang gibt es zwischen Rügen und LNG?"},#, "answer": "Rügen ist ein Standort für den Import von LNG (Flüssigerdgas) nach Deutschland. LNG ist ein kryogenes Flüssigkeitsgemisch, das hauptsächlich aus Methan besteht und bei niedrigeren Temperaturen als -160°C verflüssigt wird. Es wird in speziellen Tankern transportiert und zur Stromerzeugung oder als Brennstoff für industrielle Prozesse verwendet. Der Hafen von Mukran auf Rügen wurde als LNG-Importstandort ausgewählt, da er über eine tiefgelegte Hafenstruktur verfügt, die für den Empfang von LNG-Tankern geeignet ist. Die Anlage wird derzeit entwickelt, um die Infrastruktur für den Import von LNG bereitzustellen, was eine klimaneutrale Energiequelle darstellt."},
        {"question":"Welche Neuerungen gibt es zu § 32f Absatz 9 GWB-E?"},#, "answer":"""Der Entwurf sieht eine Änderung des § 32f Absatz 9 GWB-E vor, die lautet:
#„Im Fall eines unternehmerischen Handelns gemäß Absatz 1 bis 4 hat das Unternehmen unverzüglich und spätestens innerhalb von 30 Tagen ab dem Zeitpunkt des Abschlusses des unternehmerischen Handelns eine Meldung nach § 32g Absatz 1 Satz 1 und 2 vorzunehmen.“
#Diese Änderung beinhaltet eine Reduzierung der Frist zur Vorlage einer Meldung nach § 32g GWB-E von 60 Tagen auf 30 Tage."""},
    ])

from dspy.teleprompt import BootstrapFewShot


# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot()

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset.train)

100%|██████████| 3/3 [00:09<00:00,  3.05s/it]

Bootstrapped 3 full traces after 3 examples in round 0.


{}

In [33]:
# Ask any question you like to this simple RAG program.
my_question = "How high are the interest rates for student credits of the KfW?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)


In [34]:
pred

Prediction(
    context=['Erläuterungen: Die Kreditanstalt für Wiederaufbau (KfW) gewährt im Auftrag des Bundes Studierenden als Maßnahme zur Bildungsförderung den KfW-Studienkredit grundsätzlich als Eigenmittelprogramm. Im Zuge der Corona-Pandemie wurden KfW- Studienkredite zwischen dem 1.\u2009Mai 2020 und dem 30. September 2022 aufgrund einer Sondervereinbarung zwischen KfW und BMBF zu Lasten des Bundeshaushalts für die Kreditnehmenden zinsfrei gestellt. Zudem wurde im Zuge dessen der - 23 - Leistungsfähigkeit des Bildungswesens, 3002 Nachwuchsförderung Titel Soll Soll 2023 Ist Funktion Z w e c k b e s t i m m u n g 2024 Reste 2023 20221 000 € 1 000 € 1 000 € Noch zu Titel 661 50 (Titelgruppe 50) Antragstellerkreis bis zum 31. März 2021 auf alle ausländischen Studierenden deutscher Hochschulen erweitert, auch soweit sie die bisherigen zusätzlichen Kreditbedingungen der KfW für ausländische Studierende nicht erfüllten. ', 'Zudem hat die Kreditanstalt für Wiederaufbau zum 1. April 202

In [31]:
model.inspect_history(n=1)





Answer the question with one or more whole sentences. Base your answer solely on the context. Use the same language used in the question for your answer.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: one or more whole sentences

---

Context:
[1] «Dies entspricht im Ergebnis der dänischen Tradition des Mittelnamens. Allerdings bleibt es auch für Angehörige der dänischen Minderheit dabei, dass der Name nur gemäß der Namensstruktur deutschen Rechts gebildet werden kann, nach der der Name sich aus einem oder mehreren Vornamen und einem Familiennamen zusammensetzt. Neben Vor- und Familiennamen soll es auch künftig keinen eigenständigen Mittelnamen geben, der sich nicht in diese Struktur einfügt. Wird ein Mittelname gewünscht, kann dieser nur entweder (wie schon derzeit möglich) als zweiter Vorname oder als erster Familienname eines Doppelnamens erteil